# Berechnung einer absoluten Positionierung mit Code-Messungen, Teil 2

In [1]:
%%capture
# Requirements 
# pip install git+https://github.com/GNSSpy-Project/gnsspy
# pip install pyunpack
# pip install georinex

# Libs
import gnsspy as gp
import numpy as np
import georinex
import geopandas
import matplotlib.pyplot as plt
import math
import pandas as pd

# Params
np.set_printoptions(formatter={'float': '{: 0.5f}'.format})
plt.rcParams['figure.dpi'] = 300

In [2]:
%%capture
# Dataimport
station = gp.read_obsFile("./data/ONSA0320.11O")
ephemerides = georinex.load("./data/G3_11032.PRE")
clock = gp.read_clockFile("./data/cod16212.clk")

In [3]:
# get dfs for each epoch
clock_epoch_0 = clock[clock.Epoch == "2011-02-01 00:00:00"]
clock_epoch_1 = clock[clock.Epoch == "2011-02-01 00:15:00"]
clock_epoch_2 = clock[clock.Epoch == "2011-02-01 00:30:00"]
clock_epoch_3 = clock[clock.Epoch == "2011-02-01 00:45:00"]
clock_epoch_4 = clock[clock.Epoch == "2011-02-01 01:00:00"]

epochs_clock = list((clock_epoch_0, clock_epoch_1, clock_epoch_2, clock_epoch_3, clock_epoch_4))

ephemerides_epoch_0 = ephemerides.sel(time="2011-02-01T00:00:00.000000000")
ephemerides_epoch_1 = ephemerides.sel(time="2011-02-01T00:15:00.000000000")
ephemerides_epoch_2 = ephemerides.sel(time="2011-02-01T00:30:00.000000000")
ephemerides_epoch_3 = ephemerides.sel(time="2011-02-01T00:45:00.000000000")
ephemerides_epoch_4 = ephemerides.sel(time="2011-02-01T01:00:00.000000000")

epochs_ephemerides = list((ephemerides_epoch_0, ephemerides_epoch_1, ephemerides_epoch_2, ephemerides_epoch_3, ephemerides_epoch_4))


In [77]:
# Helper funcs for satellite Positions
# Consts
omega_e = 7.292115e-5 #s^-1
c = 299792458 #m/s

def calculateSatPos(earth_fixed_coords, sat_velocities):
    earth_fixed_coords_si = earth_fixed_coords * 1000 # km to m
    sat_velocities_si = sat_velocities / 10 # dm/s to m/s
    
    tau = math.dist(np.array(station.approx_position), earth_fixed_coords_si) / c
    sat_coords = np.array(earth_fixed_coords_si).T - np.array( (sat_velocities_si + (omega_e * np.array([-earth_fixed_coords_si[1], earth_fixed_coords_si[0], 0]))) * tau)

    return sat_coords

# Helper funcs for tropospherical correction
# Rotation matrices
def ry(a): return np.matrix([[np.cos(a), 0, -np.sin(a)], [0, 1, 0], [np.sin(a), 0, np.cos(a)]])
def rz(a): return np.matrix([[np.cos(a), np.sin(a), 0], [-np.sin(a), np.cos(a), 0], [0, 0, 1]])

def calculateLatLong(earth_fixed_coords):
    x,y,z = earth_fixed_coords

    lat = math.degrees(math.atan2(z, math.sqrt(x**2 + y**2)))
    lon = math.degrees(math.atan2(y, x))

    return lat, lon

# Topo Coords
def calculateTopoCoords(stat_coord_earth_fixed, earth_fixed_coords_at_send):
    lat_s, lon_s = calculateLatLong(stat_coord_earth_fixed)
    lat_s, lon_s = math.radians(lat_s), math.radians(lon_s)

    # Calculate N, E, U
    topo_coords = ry((math.pi / 2) - lat_s) @ rz(lon_s) @ (earth_fixed_coords_at_send - stat_coord_earth_fixed).T
    n = -topo_coords[0,0]
    e = topo_coords[0,1]
    u = topo_coords[0,2]
    
    return (n, e, u)

# Zenitwinkel
def calculateZn(topo_coords):
    n, e, u = topo_coords
    return math.degrees(math.atan2(math.sqrt(n**2 + e**2), u))

# Tropo delay
def calculateTropDelay(angle):
    return 2.4 / math.cos(math.radians(angle))

# Relativistic delay
def calculateRelativistics(coord, velocity):
    velocity = velocity / 10 # dm/s to m/s
    return 2 * (coord @ velocity.T) / c

In [87]:
# Calculate Sat positions for each epoch for each satellite in Interval A
sat_coords_at_send = list()
velo_at_send = list()
for epoch in epochs_ephemerides: # for each epoch
    epoch_helper = list()
    velo_helper = list()
    for j, coord in enumerate(epoch.position): # for each coordinate
        #print("koord", coord, "velo", epoch.velocity[j])
        epoch_helper.append(calculateSatPos(coord, epoch.velocity[j])) # calculate sat position at send time
        velo_helper.append(np.array(epoch.velocity[j]))
    sat_coords_at_send.append(np.array(epoch_helper))
    velo_at_send.append(velo_helper)

# sat_coords_at_send[epoch][satellite] example epoch 4 with g25: sat_coords_at_send[4][24])

# Calculate Tropo delay for each epoch for each satellite in Interval A
tropo_delay = list()
for epoch in sat_coords_at_send: # for each epoch
    epoch_helper = list()
    for i, coords in enumerate(epoch): # for each coordinate
        epoch_helper.append(calculateTropDelay(calculateZn(calculateTopoCoords(station.approx_position, coords))))
    tropo_delay.append(np.array(epoch_helper))

#print(tropo_delay[0][24])
#print(np.array(calculateTopoCoords(np.array(station.approx_position) * 1000, sat_coords_at_send[0][24]))) # should be 0,0,0
#print(np.array(calculateZn(calculateTopoCoords(np.array(station.approx_position), sat_coords_at_send[0][24])))) # should be 63.26054

# Calculate Relativistic delay for each epoch for each satellite in Interval A
relativistic_delay = list()
for i, epoch in enumerate(sat_coords_at_send): # for each epoch
    epoch_helper = list()
    for j, coords in enumerate(epoch): # for each velocity
        epoch_helper.append(calculateRelativistics(coords, velo_at_send[i][j]))
    relativistic_delay.append(np.array(epoch_helper))

# print(relativistic_delay[0][24])

In [90]:
def calculateP3k(f1, p1, f2, p2)
    return (f1**2 * p1 - f2**2 * p2) / (f1**2-f2**2)
# Calculate a-priori clock bias for each epoch for each satellite in Interval A
a_priori_clock_bias = list()
for i, epoch in enumerate(epochs_clock): # for each epoch
    epoch_helper = list()
    for j, delta in enumerate(epoch): # for each delta
        epoch_helper.append()
    a_priori_clock_bias.append(np.array(epoch_helper))

,Epoch,DeltaTSV
G02,2011-02-01,0.000318
G03,2011-02-01,0.000670
G04,2011-02-01,0.000024
G05,2011-02-01,-0.000108
G06,2011-02-01,-0.000049
G07,2011-02-01,0.000003
G08,2011-02-01,0.000007
G09,2011-02-01,0.000059
G10,2011-02-01,-0.000065
G11,2011-02-01,-0.000124
